<a href="https://colab.research.google.com/github/aryamanpathak2022/cv_models/blob/main/Assignment1/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
!pip install opencv-python numpy argparse

In [1]:
import cv2
import numpy as np
import os
import argparse

In [51]:
def preprocess_image(image_path):
    """
    Pre-processes an image by converting it to grayscale, resizing it,
    applying Gaussian blur, and adaptive thresholding.

    Args:
        image_path (str): Path to the input image.

    Returns:
        tuple: Processed image, thresholded image, and scale factor.
    """
    # Load the image
    image = cv2.imread(image_path)
    image2=image

    if image is None:
        raise ValueError("Error loading image. Check the file path.")

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute scale factor to resize the image while maintaining aspect ratio
    max_dim = max(image.shape[:2])
    scale_factor = 1000 / max_dim if max_dim > 1000 else 1.0

    # Resize both color and grayscale images
    new_size = (int(image.shape[1] * scale_factor), int(image.shape[0] * scale_factor))
    image = cv2.resize(image, new_size)
    gray = cv2.resize(gray, new_size)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)

    # Adaptive thresholding for edge detection
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)

    return image, thresh, scale_factor

In [87]:
def detect_edges(image, thresh, scale_factor):
    """
    Detects circular contours (likely coins) in a thresholded image and
    returns the processed image with contours drawn.

    Args:
        image (numpy.ndarray): Original input image.
        thresh (numpy.ndarray): Thresholded binary image.
        scale_factor (float): Scale factor used in preprocessing.

    Returns:
        tuple: (List of detected circular contours, Processed image with contours drawn)
    """
    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected_circles = []

    for cnt in contours:
        perimeter = cv2.arcLength(cnt, True)  # Contour perimeter
        area = cv2.contourArea(cnt)  # Contour area

        if perimeter > 0:
            circular = (4 * np.pi * area) / (perimeter ** 2)  # Measure circularity

            # Check if the shape is roughly circular and meets the minimum area condition
            if 0.7 < circular < 1 and area > 450 * (scale_factor ** 2):
                detected_circles.append(cnt)

    # Draw detected circular contours on a copy of the original image
    processed_image = image.copy()
    cv2.drawContours(processed_image, detected_circles, -1, (0, 255, 0), 2)

    return detected_circles, processed_image



In [99]:

def segmentation(image, thresh, circle_coins):
    """
    Segments circular objects (likely coins) from the image using contours and saves the result.

    Args:
        image (numpy.ndarray): Original input image.
        thresh (numpy.ndarray): Thresholded binary image.
        circle_coins (list): List of detected circular contours.

    Returns:
        numpy.ndarray: The segmented image with coins on a black background.
    """
    if not circle_coins:
        print("No coins detected for segmentation.")
        return None

    # Create an empty mask
    mask = np.zeros_like(thresh, dtype=np.uint8)

    # Draw filled contours (coins) on the mask
    cv2.drawContours(mask, circle_coins, -1, 255, thickness=cv2.FILLED)

    # Apply mask to the original image to extract coins
    segmented = cv2.bitwise_and(image, image, mask=mask)

    # Create a black background with the same shape as the image
    processed_img = np.zeros_like(image, dtype=np.uint8)

    # Overlay segmented coins on the black background
    processed_img[mask == 255] = segmented[mask == 255]




    return processed_img

In [97]:

def extract_each_coin(image, circle_coins,):
    """
    Extracts individual circular objects (coins) from the image.

    Args:
        image (numpy.ndarray): Original input image.
        circle_coins (list): List of detected circular contours.

    Returns:
        list: List of extracted coin images as numpy arrays.
    """
    segmented_coins = []

    if not circle_coins:
        print("No coins detected for extraction.")
        return segmented_coins

    for i, cnt in enumerate(circle_coins):
        # Find the minimum enclosing circle for each coin
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        center = (int(x), int(y))
        radius = int(radius)

        # Create a circular mask for extraction
        mask = np.zeros_like(image, dtype=np.uint8)
        cv2.circle(mask, center, radius, (255, 255, 255), -1)

        # Apply mask to extract the coin
        coin_segment = cv2.bitwise_and(image, mask)

        # Define cropping coordinates ensuring they stay within image bounds
        x1, y1 = max(center[0] - radius, 0), max(center[1] - radius, 0)
        x2, y2 = min(center[0] + radius, image.shape[1]), min(center[1] + radius, image.shape[0])

        # Crop the extracted coin area
        coin_segment = coin_segment[y1:y2, x1:x2]

        # Append to the list
        segmented_coins.append(coin_segment)


    return segmented_coins

In [98]:
# Take user input for paths
input_path = input("Enter the path to the input image: ")
output_path = input("Enter the directory to save the processed images: ")

# Ensures the output path is a directory
if not os.path.isdir(output_path):
    os.makedirs(output_path, exist_ok=True)

# Preprocess the image
image, thresh, sf = preprocess_image(input_path)

# Detect circular edges (coins)
circle_coins, processed_img = detect_edges(image, thresh, sf)

# Save the edges in the image
output_file = os.path.join(output_path, "edges_detected.jpg")
cv2.imwrite(output_file, processed_img)

# Segment detected coins
processed_img=segmentation(image, thresh, circle_coins)

# Save the processed image
output_file = os.path.join(output_path, "segments.jpg")
cv2.imwrite(output_file,processed_img)


# Extract each coin
segmented_coins = extract_each_coin(image, circle_coins)

# Save the cropped images of individual coins
for i in segmented_coins:
    output_file = os.path.join(output_path, f"coin_{i}.jpg")
    cv2.imwrite(output_file, i)

# Count and print the number of detected coins
coin_count = count_coin(segmented_coins)


print(f'{coin_count} coins detected')

Enter the path to the input image: test_x.jpg
Enter the directory to save the processed images: ./
6 coins detected
